In [ ]:

import itertools
import math
import random
from pathlib import Path

import codetiming as ct
import numpy as np
import scipy.signal as spsig
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px


In [ ]:
def numpy_print(txt, array):
    print(f"{txt}{np.array2string(array, precision=3, prefix=txt)}")

In [ ]:
def matrix_consensus(A, X0, eps):
    n = len(X0)
    Xo = np.zeros((n, 1))
    Xn = X0.copy()

    L = np.diag(A.sum(1)) - A
    W = np.eye(n) - eps * L

    steps = 0
    while not np.allclose(Xo, Xn, atol=0.001, rtol=0) and not np.any(np.isnan(Xn)):
        Xo = Xn
        Xn = W @ Xo
        steps += 1

    # print(f"Average initial value = {avg}\nAverage final value = {np.mean(Xn)}")

    return Xn, steps


In [ ]:
def loop_consensus(A, X0, eps):
    n = len(X0)
    Xo = np.zeros((n, 1))
    Xn = X0.copy()

    steps = 0
    while not np.allclose(Xo, Xn, atol=0.1, rtol=0) and not np.any(np.isnan(Xn)):
        Xo = Xn.copy()
        for i in range(n):
            total = 0
            for j in range(n):
                total += A[i][j] * (Xo[j] - Xo[i])
            Xn[i] = Xo[i] + eps * total
        steps += 1

    # print(f"Average initial value = {avg}\nAverage final value = {np.mean(Xn)}")

    return Xn, steps

In [ ]:
def run_consensus(A):
    num_nodes = A.shape[0]
    counts = dict(zip(*np.unique(A, return_counts=True)))
    num_edges = counts[1] / 2
    if num_nodes in initials:
        X0 = initials[num_nodes]
    else:
        X0 = np.random.rand(num_nodes, 1) * 100
        initials[num_nodes] = X0

    L = np.diag(A.sum(1)) - A
    lambdas, vectors = np.linalg.eigh(L)
    lambdas.sort()
    eps = 2 / (lambdas[1] + lambdas[-1])

    Xm, km = matrix_consensus(A, X0, eps)

    if np.isclose(lambdas[1], lambdas[-1]):
        beta = np.nan
    else:
        beta = 1 / (1 - eps * lambdas[1])

    return round(beta, 5), km, num_nodes


In [ ]:

# MAIN

x = []
y = []
c = []

data_folder = Path.cwd() / '../Data'

initials = {}

# with open(data_folder / 'RandomGraphs.npz', 'rb') as stream:
#     graphs = np.load(stream)
#
#     for g in graphs:
#         beta, k_final, additional = run_consensus(A=graphs[g])
#         x.append(beta)
#         y.append(k_final)
#         c.append(additional)

for i in range(3, 8):
    with open(data_folder / f'UniqueGraphs_{i}.npz', 'rb') as stream:
        graphs = np.load(stream)

        for g in graphs:
            beta, k_final, additional = run_consensus(A=graphs[g])
            x.append(beta)
            y.append(k_final)
            c.append(additional)

x = np.array(x)
x[np.isnan(x)] = np.nanmax(x)
y = np.array(y)
c = np.array(c)
inds = x.argsort()
y = y[inds]
x = x[inds]
c = c[inds]

In [ ]:
t = np.linspace(min(x), max(x), 100)
step = t[1] - t[0]

env_x = []
env_y = []
w = 0.5

# for v in t:
#     idx = np.where((x >= v - w * step) & (x <= v + w * step))[0]
#     if idx.size:
#         env_y.append(np.max(y[idx]))
#         env_x.append(v)


local_max = 0
for xi, yi in zip(reversed(x), reversed(y)):
    if yi > local_max:
        local_max = yi
        env_y.append(yi)
        env_x.append(xi)

env_x.reverse()
env_y.reverse()

import scipy.io
scipy.io.savemat(data_folder / 'data.mat', dict(x=env_x, y=env_y))

t1 = go.Scatter(x=x, y=y, mode='markers', name='Data',
                text=c,
                marker=dict(color=c,
                            colorscale='Viridis',
                            showscale=True)
                )

t2 = go.Scatter(x=env_x, y=env_y, mode='lines', name='Envelope')

layout = go.Layout(title="Convergence time depending on beta",
                   width=800,
                   height=600,
                   showlegend=False,
                   scene=dict(xaxis=dict(dict(title='beta')),
                              yaxis=dict(dict(title='num_steps'))),
                   margin=dict(t=100),
                   hovermode='closest')


fig = go.Figure(data=[t1, t2], layout=layout)
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()